In [1]:
import os
os.environ["OPENAI_API_KEY"] = "your key"

In [2]:
from langchain.vectorstores import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader

# Load data from a text file
loader = TextLoader("GradAscent.txt")  # Replace with your knowledge base
documents = loader.load()

# Split data into manageable chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
docs = text_splitter.split_documents(documents)

# Create a vector store with embeddings
embeddings = OpenAIEmbeddings()
vector_store = FAISS.from_documents(docs, embeddings)

# Initialize a retriever for querying the vector store
retriever = vector_store.as_retriever(search_type="similarity", search_k=3)

C:\Users\tksen\AppData\Local\Temp\ipykernel_10512\1313387204.py:15: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings()


In [29]:
import requests
from langchain.agents import Tool

# Function to get latitude & longitude from a location name
def get_lat_lon(location: str):
    geo_url = "https://nominatim.openstreetmap.org/search"
    params = {"q": location, "format": "json"}
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"}

    try:
        response = requests.get(geo_url, params=params, headers=headers)
        data = response.json()

        if data:
            return float(data[0]["lat"]), float(data[0]["lon"])
        else:
            return None, None  # Location not found

    except Exception as e:
        return None, None  # Handle API failure

# Function to fetch real-time weather using Open-Meteo API
def get_weather(location: str):
    lat, lon = get_lat_lon(location)

    if lat is None or lon is None:
        return f"Could not find location: {location}. Please try another."

    weather_url = "https://api.open-meteo.com/v1/forecast"
    params = {
        "latitude": lat,
        "longitude": lon,
        "current_weather": True
    }

    try:
        response = requests.get(weather_url, params=params)
        data = response.json()

        weather = data.get("current_weather", {})
        if weather:
            temperature = weather.get("temperature")
            wind_speed = weather.get("windspeed")
            return (f"The current weather in {location} is {temperature}°C "
                    f"with wind speed of {wind_speed} km/h.")
        else:
            return f"Weather data unavailable for {location}."

    except Exception as e:
        return f"Error fetching weather: {e}"

# Define the weather tool for the agent
weather_tool = Tool(
    name="Weather Tool",
    func=get_weather,
    description="Fetches live weather information for any given location."
)

# Test the fix
print(get_lat_lon("Chennai"))  # Should return valid latitude and longitude
print(get_weather("Chennai"))  # Should return actual weather data


(13.0836939, 80.270186)
The current weather in Chennai is 26.0°C with wind speed of 3.8 km/h.


In [30]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

# Initialize the LLM
llm = ChatOpenAI(model="gpt-4o-mini")

# Create the Retrieval QA chain
retrieval_qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    return_source_documents=True
)

In [31]:
from langchain.agents import initialize_agent, Tool, AgentType

# Combine tools and retrieval chain
def document_retrieval(query: str):
    return retrieval_qa_chain({"query": query})["result"]

tools = [
    Tool(name="Document Retrieval", func=document_retrieval, description="Retrieve knowledge from the document database."),
    weather_tool
]


# Initialize the agent
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

In [32]:
def chatbot_agentic_rag():
    print("Agentic RAG  is running! Type 'exit' to terminate.")
    while True:
        user_query = input("You: ")
        if user_query.lower() == "exit":
            print("Chatbot session terminated.")
            break
        try:
            response = agent.run(user_query)
            print(f"Bot: {response}")
        except Exception as e:
            print(f"Error: {e}")

In [33]:
chatbot_agentic_rag()

Agentic RAG Chatbot is running! Type 'exit' to quit.


You:  Where is the headquartes of greengrow and what is the weather out there now




> Entering new AgentExecutor chain...
I need to find out the location of Greengrow's headquarters first, and then I can check the current weather for that location. I'll start by retrieving information about Greengrow's headquarters.

Action: Document Retrieval  
Action Input: "Greengrow headquarters location"  
Observation: GreenGrow Innovations is headquartered in Portland, Oregon.
Thought:I have found the location of Greengrow's headquarters, which is in Portland, Oregon. Now, I will check the current weather for Portland.

Action: Weather Tool  
Action Input: "Portland, Oregon"  
Observation: The current weather in Portland, Oregon is 7.3°C with wind speed of 12.5 km/h.
Thought:I now know the final answer.  
Final Answer: The headquarters of Greengrow is located in Portland, Oregon, and the current weather there is 7.3°C with a wind speed of 12.5 km/h.

> Finished chain.
Bot: The headquarters of Greengrow is located in Portland, Oregon, and the current weather there is 7.3°C with

You:  exit


Chatbot session ended.
